<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/fi_whisper_base_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-emki7u06
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-emki7u06
  Resolved https://github.com/huggingface/transformers to commit bd50402b56980ff17e957342ef69bd9b0dd45a7b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8056436 sha256=c3c789c821243a72d8443f8f334f5c7ed9ec131a99ebc1f7946164ae388e49bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-9isjpthi/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [5]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [6]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "fi", split="validation")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 718
})

In [9]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [22]:
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 36.0MiB/s]


In [23]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [24]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [25]:
text

,0
0,kysyisinkin korkealta edustajalta voitaisiinko...
1,työtä on tehtävä ja jatkettava.
2,jos parlamentin turvallisuus on niin heikko et...
3,arvoisa puhemies pitkän prosessin jälkeen euro...
4,hänhän muistutti meitä juuri siitä kuinka tärk...
...,...
713,puututaan ja ne pyritään estämään.
714,siksi ryhmäni päättää lopullisesti kannastaan ...
715,se laadukkuus tarkoittaa sitä että nämä toimet...
716,rajavalvonta on turkin kompastuskivi. turkin j...


In [26]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [27]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,female,kysyisinkin korkealta edustajalta voitaisiinko...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, -9.1552734375e-05, 0.0001220...",16000
1,male,työtä on tehtävä ja jatkettava.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.012664794921875, -0.0162353515625, -0.0177...",16000
2,male,jos parlamentin turvallisuus on niin heikko et...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0025634765625, -0.00152587890625, -0.00082...",16000
3,female,arvoisa puhemies pitkän prosessin jälkeen euro...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000457763671875, 0.0003662109375, 0.000244...",16000
4,female,hänhän muistutti meitä juuri siitä kuinka tärk...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.165618896484375, -0.181854248046875, -0.14...",16000
...,...,...,...,...,...
713,male,puututaan ja ne pyritään estämään.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00421142578125, 0.001953125, 0.000549316406...",16000
714,male,siksi ryhmäni päättää lopullisesti kannastaan ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0001220703125, -0.000732421875, 0.00021362...",16000
715,female,se laadukkuus tarkoittaa sitä että nämä toimet...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0015869140625, -0.002349853515625, -0.0022...",16000
716,male,rajavalvonta on turkin kompastuskivi. turkin j...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002227783203125, 0.0147705078125, -0.00399...",16000


In [28]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
1,male,työtä on tehtävä ja jatkettava.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.012664794921875, -0.0162353515625, -0.0177...",16000
2,male,jos parlamentin turvallisuus on niin heikko et...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0025634765625, -0.00152587890625, -0.00082...",16000
5,male,kolmas ja viimeinen kommenttini puhemies eräät...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000762939453125, 0.00238037109375, 0.003631...",16000
7,male,siksi äänestin tässä ryhmän ehdotusta vastaan ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-9.1552734375e-05, -0.000518798828125, -0.002...",16000
8,male,aika ei valitettavasti riitä vastaamaan kaikki...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000152587890625, 0.000762939453125, 0.00048...",16000
...,...,...,...,...,...
712,male,meidän on osoitettava myötätuntoa mutta myötät...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001434326171875, 0.0008544921875, 0.0033569...",16000
713,male,puututaan ja ne pyritään estämään.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00421142578125, 0.001953125, 0.000549316406...",16000
714,male,siksi ryhmäni päättää lopullisesti kannastaan ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0001220703125, -0.000732421875, 0.00021362...",16000
716,male,rajavalvonta on turkin kompastuskivi. turkin j...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002227783203125, 0.0147705078125, -0.00399...",16000


In [29]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
0,female,kysyisinkin korkealta edustajalta voitaisiinko...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, -9.1552734375e-05, 0.0001220...",16000
3,female,arvoisa puhemies pitkän prosessin jälkeen euro...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000457763671875, 0.0003662109375, 0.000244...",16000
4,female,hänhän muistutti meitä juuri siitä kuinka tärk...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.165618896484375, -0.181854248046875, -0.14...",16000
6,female,ammatillisen tulevaisuuden ollessa epävarma nu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000885009765625, 0.000579833984375, 0.00048...",16000
13,female,monissa euroopan maissa heiltä on kielletty te...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0123291015625, 0.003692626953125, -0.011688...",16000
...,...,...,...,...,...
698,female,te sanoitte että puolassa on laillisesti valit...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00018310546875, -0.000335693359375, -3.051...",16000
701,female,arvoisa puhemies arvoisa herra neuvoston puhee...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00286865234375, -0.000396728515625, 0.0, 0....",16000
709,female,kesäaikaan siirtymisellä on kuitenkin tutkitus...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003143310546875, 0.000152587890625, 0.0039...",16000
711,female,äitinsä kanssa olisi myös voinut joutua tälle ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.03118896484375, -0.03173828125, -0.0287475...",16000


In [30]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="fi")
    return result["text"]

In [31]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [32]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.4
Processed 2 files, WER: 0.5555555555555556
Processed 3 files, WER: 0.45161290322580644
Processed 4 files, WER: 0.5454545454545454
Processed 5 files, WER: 0.6060606060606061
Processed 6 files, WER: 0.84
Processed 7 files, WER: 0.75
Processed 8 files, WER: 0.4666666666666667
Processed 9 files, WER: 0.42857142857142855
Processed 10 files, WER: 0.4
Processed 11 files, WER: 0.5714285714285714
Processed 12 files, WER: 0.8
Processed 13 files, WER: 0.46153846153846156
Processed 14 files, WER: 0.7777777777777778
Processed 15 files, WER: 0.4444444444444444
Processed 16 files, WER: 0.4166666666666667
Processed 17 files, WER: 0.6666666666666666
Processed 18 files, WER: 0.5333333333333333
Processed 19 files, WER: 0.3333333333333333
Processed 20 files, WER: 0.6
Processed 21 files, WER: 0.375
Processed 22 files, WER: 0.4444444444444444
Processed 23 files, WER: 0.4444444444444444
Processed 24 files, WER: 0.5
Processed 25 files, WER: 0.7857142857142857
Processed 26 files, WE

In [33]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.65
Processed 2 files, WER: 0.5172413793103449
Processed 3 files, WER: 0.6417910447761194
Processed 4 files, WER: 0.375
Processed 5 files, WER: 1.0
Processed 6 files, WER: 0.4666666666666667
Processed 7 files, WER: 0.5
Processed 8 files, WER: 0.4
Processed 9 files, WER: 0.4166666666666667
Processed 10 files, WER: 0.2222222222222222
Processed 11 files, WER: 0.5
Processed 12 files, WER: 0.9230769230769231
Processed 13 files, WER: 0.42105263157894735
Processed 14 files, WER: 0.5294117647058824
Processed 15 files, WER: 0.6666666666666666
Processed 16 files, WER: 0.5
Processed 17 files, WER: 0.5862068965517241
Processed 18 files, WER: 0.5416666666666666
Processed 19 files, WER: 0.47619047619047616
Processed 20 files, WER: 0.43333333333333335
Processed 21 files, WER: 0.6923076923076923
Processed 22 files, WER: 0.8666666666666667
Processed 23 files, WER: 0.4166666666666667
Processed 24 files, WER: 0.4482758620689655
Processed 25 files, WER: 0.6428571428571429
Processe